In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
with open('data/clean.json') as json_data:
    data = json.load(json_data)
len(data)

In [ ]:
f = plt.figure(figsize=(10,10))
for i,sample in enumerate(data[:4]):
    plt.subplot(4,1,i+1)
    img = mpimg.imread("data/lfw/"+sample["image"].replace('bmp',"jpg"))
    plt.imshow(img)
    plt.text(300,50,"\n".join([desc['text'] for desc in sample['descriptions']]))
    plt.axis('off')

## Pretrained models

In [ ]:
import PIL.Image
import transparent_latent_gan.src.model.cnn_face_attr_celeba as cnn_face
from transparent_latent_gan.src.tl_gan import feature_axis
import glob, os,pickle
import transparent_latent_gan.src.tl_gan.feature_celeba_organize as feature_celeba_organize
import importlib

In [ ]:
model = cnn_face.create_cnn_model(path_celeba_att="transparent_latent_gan/data/raw/celebA_annotation/list_attr_celeba.txt")
model.load_weights(cnn_face.get_list_model_save(path_model_save = 'transparent_latent_gan/asset_model/cnn_face_attr_celeba')[0])

In [ ]:
path_feature_direction = './transparent_latent_gan/asset_results/pg_gan_celeba_feature_direction_40'

pathfile_feature_direction = glob.glob(os.path.join(path_feature_direction, 'feature_direction_*.pkl'))[-1]

with open(pathfile_feature_direction, 'rb') as f:
    feature_direction_name = pickle.load(f)

feature_direction = feature_direction_name['direction']
feature_name = feature_direction_name['name']
num_feature = feature_direction.shape[1]


importlib.reload(feature_celeba_organize)
feature_name = feature_celeba_organize.feature_name_celeba_rename
feature_direction = feature_direction_name['direction'] * feature_celeba_organize.feature_reverse[None, :]

len_z = 512

## Extracting features, single image

In [ ]:
list_img_batch = []
img = PIL.Image.open("data/lfw/"+data[215]["image"].replace('bmp',"jpg")).resize((128,128),PIL.Image.BICUBIC)
img = np.asarray(img)
plt.imshow(img)
plt.axis('off')
list_img_batch.append(img)

img_batch = np.stack(list_img_batch, axis=0)
x = cnn_face.preprocess_input(img_batch)
y = model.predict(x, batch_size=1)


In [ ]:
list(zip(y[0], feature_name))

In [ ]:
# modification to get more diverse results
dont_change =  ['Male','Hairline','Big_lips','Big_nose','Narrow_Eyes','Skin_Tone','Smiling','Mouth_Open','Age']
np.array([(x if x > 0 or feature_name[i] in dont_change else 0) for i,x in enumerate(y[0])])
# uncomment next to see unmodified 
# y    

In [ ]:
list_img_batch = []
for d in data:   
    img = np.asarray(PIL.Image.open("data/lfw/"+d["image"].replace('bmp',"jpg")).resize((128,128),PIL.Image.BICUBIC)) # uncropped
    # PIL.Image.open("data/lfw/"+data[215]["image"].replace('bmp',"jpg")).crop((125-64,125-64,125+64,125+64)) # cropped
    list_img_batch.append(img)

In [ ]:
img_batch = np.stack(list_img_batch, axis=0)
x = cnn_face.preprocess_input(img_batch)
y = model.predict(x, batch_size=1)

In [ ]:
y.shape

In [ ]:
target_loc = "NN_Approach_results/plain/"

In [ ]:
# modification for more diverse results
new_y =[] 
for vec in y:
    v = [(x if x > 0 or feature_name[i] in dont_change else 0) for i,x in enumerate(vec)]
    new_y.append(v)
    np.savetxt(target_loc + "image_features_modified.txt",np.array(new_y))

In [ ]:
np.savetxt(target_loc+ "image_features_modified.txt",np.array(y) )

The output files can be used in kaggle kernel that runs tl-GAN demo to predict images for each vector. The necessary code is in cell below.
Instructions for running tl-gan demo [https://github.com/SummitKwan/transparent_latent_gan]

In [ ]:
### copy this cell to tl-GAN demo krnel to get output
!mkdir "/kaggle/working/generated_faces"
feat_vecs = np.loadtxt("../../../simple-feature-extraction-results/simple_feats_v4.txt") # change to location of your dataset
for i, feature_vec in enumerate(feat_vecs[:10]):
    z_sample = feature_direction.dot(feature_vec) #np.random.randn(len_z)
    # the generated image using this noise patter
    x_sample = generate_image.gen_single_img(z=z_sample, Gs=Gs)
    imgObj = PIL.Image.fromarray(x_sample)
    imgObj.save("/kaggle/working/generated_faces/test_{}.png".format(i), format='PNG')
shutil.make_archive("/kaggle/working/generated_faces", 'zip', "/kaggle/working/generated_faces")
!rm -r /kaggle/working/generated_faces

## Preparing data and training the model
**Won't give reasonable results because fearure extraction fails**

In [ ]:
out_features = np.loadtxt("image_features_modified.txt")

In [ ]:
out_features.shape

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
data[0]['descriptions'][0]['text']

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data

embed = load_vectors("wiki-news-300d-1M-subword.vec")

In [ ]:
sent_to_img = []
sentences = []
for i,sample in enumerate(data):
    """"" separate descriptions 
    for desc in sample['descriptions']:
        text = word_tokenize(desc['text'].lower()) 
        sent_to_img.append(i)
        sentences.append(text)
    """"" 
    text = word_tokenize(" ".join([desc['text'].lower() for desc in sample['descriptions']]))
    sent_to_img.append(i)
    sentences.append(text)
    
    

In [ ]:
vocab = list(set([s for sent in sentences for s in sent]))

In [ ]:
weights_matrix = np.zeros((len(vocab)+1, 300))
for i, word in enumerate(["<pad>"]+vocab):
    try: 
        weights_matrix[i] = embed[word]
    except KeyError:
        weights_matrix[i] = np.random.rand(300) 

In [ ]:
index = {}
for i, word in enumerate(["<pad>"]+vocab):
    index[word] = i

In [ ]:
X = np.array([[index[w] for w in txt if w in index] for txt in sentences])
Y = np.array([out_features[i] for i in sent_to_img])
val_idx = np.random.randint(0, X.shape[0], 20)
train_idx = np.full(X.shape[0],True)
train_idx[val_idx]=False
X_train = X[train_idx]
Y_train = Y[train_idx]
X_val = X[val_idx]
Y_val = Y[val_idx]

In [ ]:
from allennlp.data.fields import ArrayField
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from typing import Iterator, List, Dict

class InstanceMaker(DatasetReader):
    def __init__(self) -> None:
        super().__init__(lazy=False)

    def vec_to_instance(self, description: List[int], features: List[float] = np.zeros(1)) -> Instance:
        desc_field =  ArrayField(np.array(description,int))
        fields = {"description": desc_field, "num_tokens": len(description)}

        if features.any():
            features_field = ArrayField(np.array(features))
            fields["features"] = features_field

        return Instance(fields)
    
    def _read(self, encoded_set) -> Iterator[Instance]:
        for desc, feat in encoded_set:
            yield self.vec_to_instance(desc,feat)
            
            

class TextToFeature(nn.Module):

    def __init__(self, hidden_size, out_size, weights_matrix):
        super(TextToFeature, self).__init__()        
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix))
        self.embedding.weight.requires_grad = False
        
        self.lstm = nn.LSTM(weights_matrix.shape[1], hidden_size)
        self.vec2out = nn.Linear(hidden_size, out_size)
        
    def forward(self, x):  
        embedded = self.embedding(x)
        o, hidden = self.lstm(embedded.view(embedded.size()[1], embedded.size()[0], -1))
        hidden = hidden[0].view(hidden[0].size()[1], -1)
        out = self.vec2out(hidden)
        return torch.tanh(out)
    


In [ ]:
maker = InstanceMaker()
train_set = maker.read(zip(X_train,Y_train))
val_set = maker.read(zip(X_val,Y_val))

In [ ]:
train_set.sort(key = lambda x: x['num_tokens']) # to group with similar length
train_set.reverse()
val_set.sort(key = lambda x: x['num_tokens']) # to group with similar length
val_set.reverse()

In [ ]:
import torch.optim as optim
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

hidden_size = 800

model = TextToFeature(hidden_size, 40, weights_matrix)
criterion = torch.nn.functional.hinge_embedding_loss
optimizer = optim.Adam(model.parameters(), lr=0.008, weight_decay=0.001)

In [ ]:
epochs = 30
batch_size = 64
num_instances = len(train_set)
tr_losses = []
val_losses = []
best_v = 100

for i in range(epochs): 
    losses = 0
    model.train()
    for j in range(0,num_instances,batch_size):       
        batch = train_set[j:j+batch_size]
        pad_len=batch[0]['num_tokens'] # pad based on longst in batch
        np.random.shuffle(batch) 
        
        X = torch.stack([element['description'].as_tensor({'dimension_0':pad_len}).long() for element in batch])
        Y = torch.stack([element['features'].as_tensor({'dimension_0':40}) for element in batch])
        
        optimizer.zero_grad()
        out = model.forward(X) 
        loss = criterion((Y-out).abs(), torch.tensor(np.ones(Y.shape)) )
        losses += loss.item()
        loss.backward()                                   
        optimizer.step()
        
        
    model.eval()
    pad_len = val_set[0]['num_tokens']
    X = torch.stack([element['description'].as_tensor({'dimension_0':pad_len}).long() for element in val_set])
    Y = torch.stack([element['features'].as_tensor({'dimension_0':40}) for element in val_set])
    
    out = model.forward(X)      
    loss = criterion((Y - out).abs(), torch.tensor(np.ones(Y.shape)))
    if loss.item() < best_v:
        best_v = loss.item()
        torch.save(model.state_dict(), "best_model.pt")
    tr_losses.append(losses/np.ceil(num_instances/batch_size))
    val_losses.append(loss.item())                                    
    print("eval loss:",loss)
    print("avg training loss:",tr_losses[-1])

In [ ]:
plt.plot(np.arange(len(tr_losses)),tr_losses,label="train")
plt.plot(np.arange(len(tr_losses)),val_losses,label="val")
plt.legend()
plt.show()

In [ ]:
model = TextToFeature(hidden_size, 40, weights_matrix)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
pad_len = val_set[0]['num_tokens']
X = torch.stack([element['description'].as_tensor({'dimension_0':pad_len}).long() for element in val_set])
out = model.forward(X) 